In [1]:
#make the imports
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#setup browser variable for Chrome, making sure splinter is setup and 
#updated
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\Dell\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [3]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [4]:
#visit the url
browser.visit(url)

In [12]:
#after inspecting, get to the info for the latest article with soup
html = browser.html
soup = bs(html, 'html.parser')
content_titles = soup.find_all('div', class_='content_title')
#for some reason it says the first article is Mars Now, though it is
#actually NASA's Mars Fleet Lies Low with Sun Between Earth and Red Planet
#so I'll go deeper into content_titles to get that

In [13]:
print(latest)

[<div class="content_title">Mars Now</div>, <div class="content_title"><a href="/news/9051/nasas-mars-fleet-lies-low-with-sun-between-earth-and-red-planet/" target="_self">NASA's Mars Fleet Lies Low with Sun Between Earth and Red Planet</a></div>, <div class="content_title"><a href="/news/9048/nasas-perseverance-rover-cameras-capture-mars-like-never-before/" target="_self">NASA's Perseverance Rover Cameras Capture Mars Like Never Before</a></div>, <div class="content_title"><a href="/news/9046/nasas-insight-finds-three-big-marsquakes-thanks-to-solar-panel-dusting/" target="_self">NASA's InSight Finds Three Big Marsquakes, Thanks to Solar-Panel Dusting</a></div>, <div class="content_title"><a href="/news/9043/nasas-ingenuity-helicopter-captures-a-mars-rock-feature-in-3d/" target="_self">NASA's Ingenuity Helicopter Captures a Mars Rock Feature in 3D </a></div>, <div class="content_title"><a href="/news/9041/take-a-3d-spin-on-mars-and-track-nasas-perseverance-rover/" target="_self">Take a

In [16]:
#since I used find_all() I got a list back, and can index for the second
#value, which I know is the article I'm look for from glancing at the site
further = latest[1]
further

<div class="content_title"><a href="/news/9051/nasas-mars-fleet-lies-low-with-sun-between-earth-and-red-planet/" target="_self">NASA's Mars Fleet Lies Low with Sun Between Earth and Red Planet</a></div>

In [17]:
#now get just the text for the title
news_title = further.text
news_title

"NASA's Mars Fleet Lies Low with Sun Between Earth and Red Planet"

In [21]:
#next go in and get the text for the description of the article
news_p = soup.find('div', class_='article_teaser_body').text
print(news_p)

The missions will continue collecting data about the Red Planet, though engineers back on Earth will stop sending commands to them until mid-October.
